In [5]:
####################
#set up packages 
####################
import ee                     
ee.Initialize()
import pandas as pd
import geemap.eefolium as emap
import csv
import time
from datetime import date
import numpy as np
import os, sys

In [6]:
####################
#User defined functtions
####################
##create not overlapping zones along the hurricane track
def createBuffer(dist1,dist2,vector):
    buffer = vector.buffer(dist1).difference(vector.buffer(dist2),ee.ErrorMargin(1))
    return buffer  

In [17]:
if False:
    files = os.listdir('C:/Users/Y/Desktop/CHES/loss/data/ibtracs/onehr_landing/LandingSegment50kmInland_since2013/')
    #print(len(files),'files: ',files)
    file = 'h5_Maria2017_2017260N12310_2017092009.csv'
    data0 = pd.read_csv('C:/Users/Y/Desktop/CHES/loss/data/ibtracs/onehr_landing/LandingSegment50kmInland_since2013/'+ file) 
    data0
    Hid = data0.iloc[0]['SID']
    landingYear = file.split('_')[3].split('.')[0][0:4]
    landingMonth = file.split('_')[3].split('.')[0][4:6]
    Hyear = data0.iloc[0]['Year']
    Hname = data0.iloc[0]['Name']
    Hname = ''.join(filter(str.isalnum, Hname)).capitalize()
    #print(landingYear)
    #print(landingMonth)
    #print(Hyear)
    #print(Hname)
    data = data0[['Longitude','Latitude']]
    #coors = data.values.tolist()
    #track = ee.Feature(ee.Geometry.LineString(coors),{'Name':Hname, 'Year': str(Hyear)})
    coors = data.values.tolist()[1]
    track = ee.Feature(ee.Geometry.Point(coors),{'Name':Hname, 'Year': str(Hyear)})
    trackBuffer = track.buffer(10000)  
    print(coors)

In [22]:
coors = [-65.9, 18.0]
Hyear = 2017
Hname = 'Maria'
track = ee.Feature(ee.Geometry.Point(coors),{'Name':Hname, 'Year': str(Hyear)})
trackBuffer = track.buffer(10000)  
print(coors)
print(track.getInfo()['properties'])

[-65.9, 18.0]
{'Name': 'Maria', 'Year': '2017'}


In [18]:
#create zones along the track
buffer10k = track.buffer(10000)
buffer20k = createBuffer(20000,10000,track)
buffer30k = createBuffer(30000,20000,track)
buffer40k = createBuffer(40000,30000,track)
buffer50k = createBuffer(50000,40000,track)

In [23]:
NL_pre = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
           .filterDate(str(Hyear-1)+'-01-01', str(Hyear-1)+'-12-31')\
           .select(['avg_rad'])\
           .first()
urban = NL_pre.gt(3)   
urban = urban.updateMask(urban.neq(0))

urban_vec = urban.clip(trackBuffer.geometry()).eq(1)\
                                  .updateMask(urban.clip(trackBuffer.geometry()).eq(1))\
                                  .reduceToVectors(geometry= trackBuffer.geometry(),
                                                    scale= 450,
                                                    geometryType= 'Polygon',
                                                    eightConnected=True,  #True or False?
                                                    maxPixels=1e13)   

In [24]:
if Hyear < 2008: 
    URyear = 2000
if Hyear >= 2009: 
    URyear = 2015   

urban = ee.ImageCollection("JRC/GHSL/P2016/SMOD_POP_GLOBE_V1")\
           .filterDate(str(URyear)+'-01-01', str(URyear)+'-12-31')\
           .select(['smod_code'])\
           .first()

HDC = urban.eq(3)
HDC = HDC.updateMask(HDC.neq(0))
HDC_vec = HDC.clip(trackBuffer.geometry())\
             .reduceToVectors(geometry= trackBuffer.geometry(),
                              scale= 1000,
                              geometryType= 'Polygon',
                              eightConnected=False,  #True or False?
                              maxPixels=1e13)   

LDC = urban.eq(2)
LDC = LDC.updateMask(LDC.neq(0))
LDC_vec = LDC.clip(trackBuffer.geometry())\
             .reduceToVectors(geometry= trackBuffer.geometry(),
                              scale= 1000,
                              geometryType= 'Polygon',
                              eightConnected=False,  #True or False?
                              maxPixels=1e13) 

DC = urban.gt(1)
DC = DC.updateMask(DC.neq(0))
DC_vec = DC.clip(trackBuffer.geometry())\
           .reduceToVectors(geometry= trackBuffer.geometry(),
                            scale= 1000,
                            geometryType= 'Polygon',
                            eightConnected=False,  #True or False?
                            maxPixels=1e13) 

In [25]:
Map = emap.Map(center=[data.iloc[0]['Latitude'],data.iloc[0]['Longitude']], zoom=8)
Map.add_basemap('ROADMAP') # Add Google Map
Map.addLayer(urban,{'min':1,'max':3},'Urban')
Map.addLayer(HDC,{},'HDC')
#Map.addLayer(HDC_vec,{'color':'#800080'},'HDC_vec')
Map.addLayer(LDC,{},'LDC')
#Map.addLayer(LDC_vec,{'color':'#008000'},'LDC_vec')
Map.addLayer(DC,{},'DC')
#Map.addLayer(DC_vec,{'color':'#0000FF'},'DC_vec')
Map.addLayer(track,{'color':'800080'},'track')
Map.addLayer(buffer10k,{'color':'FFFF00'},'Buffer 10k')
Map.addLayer(buffer20k,{'color':'FFA500'},'Buffer 20k')
Map.addLayer(buffer30k,{'color':'FFFF00'},'Buffer 30k')
Map.addLayer(buffer40k,{'color':'FFA500'},'Buffer 40k')
Map.addLayer(buffer50k,{'color':'FFFF00'},'Buffer 50k')
#Map.addLayer(urban_vec,{'color':'FF0000'},'Urban_vec')

Map.setControlVisibility(all)
Map

In [26]:
print(HDC_vec.aggregate_sum('count').getInfo())
print(LDC_vec.aggregate_sum('count').getInfo())
print(DC_vec.aggregate_sum('count').getInfo())

0
33
33


In [30]:
img_rsl = 450

yearList = list(range(2017,2018))


for yr in yearList: 
    print(yr)
    imgClt = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
               .filterDate(str(yr)+'-01-01', str(yr)+'-12-31')\
               .select(['avg_rad']) 

    indexList = ['HDC','LDC','DC']
    vecList = [HDC_vec,LDC_vec,DC_vec]

    df = pd.DataFrame(columns=(['Collection'] + ['imageIndex'] + indexList)) #create df 
    
    for v in range(0,3):
        index = indexList[v]
        vector = vecList[v] 
        print(index)
        imgClt_addedProperty = imgClt.map(lambda x: x.set({index:
                                                  x.clip(trackBuffer)\
                                                   .reduceRegion(geometry=vector,reducer=ee.Reducer.mean(),scale=img_rsl)\
                                                   .get('avg_rad')}))        
        listOfImages = imgClt_addedProperty.toList(imgClt_addedProperty.size()).getInfo()
        imageIndex = [x['properties']['system:index'] for x in listOfImages]
    
        values = [0] * len(listOfImages)
        for v in range(0,len(listOfImages)):
            if index in listOfImages[v]['properties']: 
                values[v] = listOfImages[v]['properties'][index]

   
        df['Collection'] = ["NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG"] * len(listOfImages)
        df['imageIndex'] = imageIndex 
        df[index] = values

2017
HDC
LDC
DC


In [31]:
df

,Collection,imageIndex,HDC,LDC,DC
0,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20170101,0,5.211457,5.211457
1,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20170201,0,4.503802,4.503802
2,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20170301,0,4.996159,4.996159
3,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20170401,0,4.506425,4.506425
4,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20170501,0,4.062849,4.062849
5,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20170601,0,4.245161,4.245161
6,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20170701,0,4.527360,4.527360
7,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20170801,0,4.065526,4.065526
8,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20170901,0,3.734844,3.734844
9,NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG,20171001,0,0.627824,0.627824


In [ ]:
outfile = 'C:/Users/Y/Desktop/CHES/loss/result/downloaded/' + file[0:(len(file)-4)] + '_' + 'allYears' + '_'\
          + date.today().strftime('%Y-%m-%d') + '.csv'  #set up output file
with open(outfile, 'w', newline='') as output:
    writer = csv.writer(output, delimiter=',')
    writer.writerow(['Collection'] + ['imageIndex'] + indexList)   #write row names
    for row in range(0,len(df.index)):
            writer.writerow(df.loc[row,])